Section 1 - Import Dependencies

In [1]:
#Import Libraries & Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import os
import gmaps
import scipy.stats as sts
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from config import api_key_ggle

%matplotlib inline

Store Part I results into DataFrame

Import files

In [2]:
#Load CSV containing from weatherpy out put
path = os.path.join('..','WeatherPy','Weatherhome.csv')
data_df = pd.read_csv(path)
data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ushuaia,-54.8000,-68.3000,50.00,93,40,2.30,AR,1613628061
1,Hong Kong,22.2855,114.1577,69.01,58,33,1.99,HK,1613628061
2,Rawson,-43.3002,-65.1023,64.92,54,15,6.40,AR,1613628062
3,Bathsheba,13.2167,-59.5167,77.00,65,20,25.32,BB,1613628065
4,Alamor,-4.0181,-80.0197,57.42,100,71,2.89,EC,1613628065


Humidity Heatmap

Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [3]:
# configure gmaps with API Key
gmaps.configure(api_key=api_key_ggle) 

In [4]:
#store 'Lat' and 'Lng' into locations
locations = data_df[['Lat', 'Lng']].astype(float)
#drop any NaN values in Humidity column
#hum = data_df.dropna()
humidity = data_df["Humidity"].astype(float)
#humidity = hum['Humidity'].astype(float)
#data_df = data_df.dropna()
humidity

0       93.0
1       58.0
2       54.0
3       65.0
4      100.0
       ...  
597     93.0
598     86.0
599     85.0
600     85.0
601     29.0
Name: Humidity, Length: 602, dtype: float64

Add Heatmap layer to Map

In [5]:
fig = gmaps.figure(center=(0,0), zoom_level=2)
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius = 3,)
fig.add_layer(heatmap_layer)
fig
#map_type='HYBRID',

Figure(layout=FigureLayout(height='420px'))

In [6]:
locations

,Lat,Lng
0,-54.8000,-68.3000
1,22.2855,114.1577
2,-43.3002,-65.1023
3,13.2167,-59.5167
4,-4.0181,-80.0197
...,...,...
597,53.7453,62.0708
598,65.0083,53.9113
599,43.7501,-81.7165
600,52.0333,47.3833


Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.

In [7]:
#Configure Gmaps and narrow down the DataFrame to find ideal weather conditions
#Drop any rows that don't contain all three conditions

condition_df = data_df.loc[(data_df['Max Temp']<80) & (data_df['Max Temp']>70)\
                & (data_df['Wind Speed']<10)\
                        &(data_df['Cloudiness']==0)]
condition_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
210,Kuruman,-27.4524,23.4325,72.59,43,0,7.00,ZA,1613628067
280,Vila Velha,-20.3297,-40.2925,75.20,88,0,7.92,BR,1613628055
415,Gobabis,-22.4500,18.9667,71.42,35,0,4.54,NaN,1613628119
417,Bozdar Wada,27.1828,68.6360,73.40,43,0,4.61,PK,1613628119
468,Puerto Colombia,10.9878,-74.9547,78.80,83,0,4.61,CO,1613628125
520,Quepos,9.4236,-84.1652,71.60,64,0,8.05,CR,1613628131


 Hotel Map

Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [8]:
hotel_df=condition_df
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
210,Kuruman,-27.4524,23.4325,72.59,43,0,7.00,ZA,1613628067
280,Vila Velha,-20.3297,-40.2925,75.20,88,0,7.92,BR,1613628055
415,Gobabis,-22.4500,18.9667,71.42,35,0,4.54,NaN,1613628119
417,Bozdar Wada,27.1828,68.6360,73.40,43,0,4.61,PK,1613628119
468,Puerto Colombia,10.9878,-74.9547,78.80,83,0,4.61,CO,1613628125


In [9]:
hotel_df['Hotel Name']=""


In [10]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": api_key_ggle
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_address = requests.get(base_url, params=params)

    # convert to json
    hotel_address = hotel_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_address["results"][0]["vicinity"]
        print(f"Closest hotel is {hotel_address['results'][0]['name']}")
     
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head()       


Closest hotel is Kuruman
Closest hotel is Vila Velha
Closest hotel is Gobabis
Closest hotel is Bozdar Wada
Closest hotel is Sabanilla Montecarmelo
Closest hotel is Quepos


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
210,Kuruman,-27.4524,23.4325,72.59,43,0,7.00,ZA,1613628067,Kuruman,Kuruman
280,Vila Velha,-20.3297,-40.2925,75.20,88,0,7.92,BR,1613628055,Vila Velha,Vila Velha
415,Gobabis,-22.4500,18.9667,71.42,35,0,4.54,NaN,1613628119,Gobabis,Gobabis
417,Bozdar Wada,27.1828,68.6360,73.40,43,0,4.61,PK,1613628119,Bozdar Wada,Bozdar Wada
468,Puerto Colombia,10.9878,-74.9547,78.80,83,0,4.61,CO,1613628125,Sabanilla Montecarmelo,Sabanilla Montecarmelo


In [11]:
hotel_df=hotel_df.dropna()

In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# Update with DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
210,-27.4524,23.4325
280,-20.3297,-40.2925
417,27.1828,68.6360
468,10.9878,-74.9547
520,9.4236,-84.1652


In [13]:
# Add marker layer ontop of heat map
# Display figure
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#checking number of cities limited
#condition_df.shape

(18, 9)

In [88]:
# #Filter and Create data frame to hold required values needed to pick hotel
# #Data frame created to tie the coordinates for hotel search. nearby hotels within 5000 meters. 
# clean_cond_df =condition_df[['City', 'Humidity', 'Country', 'latLng']]          
# clean_cond_df

In [89]:
# #Use Google places API to find first hotel for each city within 5000 meters

# clean_cond_df = condition [["City....
# base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
# #radius = "5000" # 1 mile proximity converted to meters

#     #location = 

# params = {
# "radius": radius,
# "query": ["hotel"],
# "type": ['Name','City','Country'],
# #"maxprice": max_price,
# "key":api_key_ggle,
# }

# for i, row in clean_cond.iterrows():
    
#     params["location"] = row[0]
#     response = requests.get(base_url, params=params).json()
#     try:
#         clean_cond.loc[i,'hotel']= pprint(response) 
#         #print(len(response['results']))
#         #print(row[0])
    
#     except:
#         clean_cond.loc[i,'hotel']= np.nan

# # = len(response['results'])
# clean_cond
#  #response.keys(). 
# #print(response)